# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов с тарифами "Смарт" и "Ультра". Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

## Откроем и изучим файл

Загрузим нужны библиотеки

In [7]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from datetime import datetime
from tqdm import tqdm

Загрузим и посмотрим на базу данных

In [8]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.head())
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разабьём данные на выборки

In [9]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

features_train, features_valid_and_test, target_train, target_valid_and_test = train_test_split(
    features, target, test_size=0.5, random_state=12345)

features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid_and_test, target_valid_and_test, test_size=0.5, random_state=12345)

In [10]:
print('Размеры наборов:')
print('Тренировочный набор - параметры (features_train) = {:.0%}'.format(len(features_train)/len(df)))
print('Тренировочный набор - целевой параметр (target_train) = {:.0%}'.format(len(target_train)/len(df)))
print('Валидационная выборка - параметры (features_valid) = {:.0%}'.format(len(features_valid)/len(df)))
print('Валидационная выборка - целевой параметр (target_valid) = {:.0%}'.format(len(target_valid)/len(df)))
print('Тестовая выборка - параметры (features_test) = {:.0%}'.format(len(features_test)/len(df)))
print('Тестовая выборка - целевой параметр (target_test) = {:.0%}'.format(len(target_test)/len(df)))

Размеры наборов:
Тренировочный набор - параметры (features_train) = 50%
Тренировочный набор - целевой параметр (target_train) = 50%
Валидационная выборка - параметры (features_valid) = 25%
Валидационная выборка - целевой параметр (target_valid) = 25%
Тестовая выборка - параметры (features_test) = 25%
Тестовая выборка - целевой параметр (target_test) = 25%


## Исследуем модели

### DecisionTreeClassifier

In [11]:
tree_model = 0
best_result = 0
value_depth = 0

start_time = datetime.now()

for depth in range(1,6):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if best_result < result:
        best_result = result
        tree_model = model
        value_depth = depth
        
print('Время поиска лучшей модели = {}'.format((datetime.now()-start_time)))

print('Правильных предсказаний - {:.2%} при максимально глубине {}'.format(best_result, value_depth))


Время поиска лучшей модели = 0:00:00.023527
Правильных предсказаний - 80.10% при максимально глубине 4


### RandomForestClassifier

In [12]:
forest_model = 0
best_result = 0
best_n_estimators = 0
best_depth = 0
best_min_samples_split = 0
best_min_samples_leaf = 0
best_criterion = ''

start_time = datetime.now()


def learn_model(criterion_var, 
                n_estimators_var, 
                max_depth_var, 
                min_samples_split_var, 
                min_samples_leaf_var):
    
    
    global forest_model
    global best_result
    global best_n_estimators
    global best_depth
    global best_min_samples_split
    global best_min_samples_leaf
    global best_criterion

    model2 = RandomForestClassifier(criterion=criterion_var,
                                    n_estimators=n_estimators_var, 
                                    max_depth=max_depth_var,
                                    min_samples_split=min_samples_split_var, 
                                    min_samples_leaf=min_samples_leaf_var,
                                    random_state=12345, 
                                    n_jobs=-1)    

    model2.fit(features_train, target_train)
    predictions_valid = model2.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)

    if best_result < result:
        best_result = result
        forest_model = model2
        best_criterion = criterion_var
        best_n_estimators = n_estimators_var
        best_depth = max_depth_var    
        best_min_samples_split = min_samples_split_var
        best_min_samples_leaf = min_samples_leaf_var

N = 5
        
for crit in ['gini', 'entropy']:
    for est in tqdm(range(1,N)):
        for depth in range(1,N):
            for mss in range(2,N):
                for msl in range(1,N):
                    learn_model(crit, est, depth, mss, msl)      

print('Время поиска лучшей модели  = {}'.format((datetime.now()-start_time)))

print('Правильных предсказаний - {:.2%} (criterion="{}"; \
n_estimator={}; max_depth={}; min_samples_split={}; \
min_samples_leaf={})'.format(best_result, 
                             best_criterion, 
                             best_n_estimators, 
                             best_depth, 
                             best_min_samples_split, 
                             best_min_samples_leaf))   

# N=5
# Время поиска лучшей модели  = 0:01:44.462451
# Правильных предсказаний - 81.72% (criterion: gini; n_estimator: 3; max_depth: 5; min_samples_split: 3; min_samples_leaf: 1)

# N=10
# Время поиска лучшей модели  = 0:02:40.708191
# Правильных предсказаний - 82.46% (criterion="entropy"; n_estimator=9; max_depth=9; min_samples_split=7; min_samples_leaf=2)

# N=15
# Время поиска лучшей модели  = 0:26:08.130099
# Правильных предсказаний - 82.96% (criterion="gini"; n_estimator=13; max_depth=9; min_samples_split=14; min_samples_leaf=3)

# N=20
# Время поиска лучшей модели  = 2:04:32.197412
# Правильных предсказаний - 83.08% (criterion="gini"; n_estimator=5; max_depth=14; min_samples_split=19; min_samples_leaf=9)

100%|██████████| 4/4 [00:01<00:00,  2.61it/s]

Время поиска лучшей модели  = 0:00:03.038321
Правильных предсказаний - 79.98% (criterion="gini"; n_estimator=3; max_depth=4; min_samples_split=2; min_samples_leaf=1)


### LogisticRegression

In [13]:
start_time = datetime.now()

model3 = LogisticRegression(random_state=12345)
model3.fit(features_train, target_train)
result = model3.score(features_valid, target_valid)

print('Время обучения модели логистической регрессии  = {}'.format((datetime.now()-start_time)))

print('Правильных предсказаний - {:.2%}'.format(result))

Время обучения модели логистической регрессии  = 0:00:00.042642
Правильных предсказаний - 76.74%


### ВЫВОД ###
|           Модель       | Accuracy, % |      Время    |
|:----------------------:|:-----------:|:------------------:|
| DecisionTreeClassifier |    80.10%   |      ~ 0.016  сек  |
| RandomForestClassifier |    83.08%   |      ~ 02:04:32    |
| LogisticRegression     |    75.50%   |      ~ 00.011 сек  |

Т.о. Случайный лес показал лучшие результаты по качеству, но хучшие по времени поиска.
Второе место (и самое оптимальное) заняла модель Решающее Дерево - быстро по времени и не сильно хуже по качеству
Третье место по качеству занимает модель логистической регрессии, но она самая быстрая



## Проверьте модель на тестовой выборке

In [14]:
predications_test = forest_model.predict(features_test)
result = accuracy_score(target_test, predications_test)

print('Правильных предсказаний на тестовой выборке - {:.2%}'.format(result))

Правильных предсказаний на тестовой выборке - 77.83%


### Вывод ###
На тестовой выборке модель (случайный лес) показала сходный (чуть ниже) результат - 78.52%, по сравнению с валидационной выборкой.
<p>
    Повторно при улучшении качества модели (за счет перебора параметров) до 83.08% на тестовой выборке эта модель показала результат ниже предыдущей модели - 78.21% (на 0.3%). Это значит я ее переучил?
    

## Проверим модел на адекватность

In [15]:
model4 = DummyClassifier(strategy='most_frequent')
model4.fit(features_train, target_train)
print(model4.score(features_valid, target_valid))

0.7101990049751243


### Вывод 
Наша модель с показателем 78.58% лучше константной модели с показателем 71%. Проверка на адекватность пройдена

# Общий вывод:
1. Модели различаются как по качеству, так и по скорости выполнения.
2. В нашем примере модель Случайный лес показала лучшие результаты в отношении качество/скорость.
3. За счет настройки гиперпараметров удалось улучшить качество модели (за счет затрат большего времени)
4. Логистическая регрессия - из 3х рассматриваемых моделей работает быстрее всех.